# Vacation Search
UC Berkeley Extension Data Analytics Boot Camp Module 6 Challenge - Part 2

## Part 2 Instructions
#### Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

### Import the Data
#### Use the CSV file created in Part 1


In [57]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [58]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("data/WeatherPy_Database.csv")


### Filter the DataFrame for minimum and maximum temperature preferences, and if the rain or snow accumulation is 0 inches or not using conditional statements. Do the following:
- Prompt the customer for the minimum temperature preference.
- Prompt the customer for the maximum temperature preference.
- Prompt the customer to answer if he or she would like it to be raining or not, using input("Do you want it to be raining? (yes/no) ").
- Prompt the customer to answer if he or she would like it to be snowing or not, using input("Do you want it to be snowing? (yes/no) ").

In [59]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [62]:
# Prompt for Max Temperatures:

min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 60
What is the maximum temperature you would like for your vacation? 75


In [63]:
# Prompt for Rain & Snow:

rain = input("Do you want it to be raining? (yes/no) ")
snow = input("Do you want it to be snowing? (yes/no) ")

Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [67]:
# Filter the dataframe based on user inputs: 4 combinations of rain & snow needed

if rain=="no" and snow=="no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & 
                                          (city_data_df["Max Temp"]>= min_temp) &
                                          (city_data_df["Rain (in)"]==0) &
                                          (city_data_df["Snow (in)"]==0)]
    
elif rain=="yes" and snow=="no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & 
                                          (city_data_df["Max Temp"]>= min_temp) &
                                          (city_data_df["Rain (in)"]>=0) &
                                          (city_data_df["Snow (in)"]==0)]    
    
elif rain=="no" and snow=="yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & 
                                          (city_data_df["Max Temp"]>= min_temp) &
                                          (city_data_df["Rain (in)"]==0) &
                                          (city_data_df["Snow (in)"]>=0)]     
    
elif rain=="yes" and snow=="yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & 
                                          (city_data_df["Max Temp"]>= min_temp) &
                                          (city_data_df["Rain (in)"]>=0) &
                                          (city_data_df["Snow (in)"]>=0)] 
    
filtered_cities_df = filtered_cities_df.dropna()


### Add the cities to a marker layer map with a pop-up marker for each city that includes:
- Hotel name
- City
- Country
- Current weather description with the maximum temperature

In [68]:
# create a hotel dataframe that copies over data from the filtered df. Lat and Lng required in order to use gmaps
hotel_df = filtered_cities_df[["City","Country","Current Description","Max Temp","Lat","Lng"]].copy()

# add hotel name column
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
6,Hailin,CN,clear sky,61.43,44.58,129.42,
7,Xichang,CN,light rain,72.68,27.90,102.26,
8,Cape Town,ZA,clear sky,71.60,-33.93,18.42,
9,Carnarvon,AU,clear sky,64.40,-24.87,113.63,
11,Rikitea,PF,clear sky,74.98,-23.12,-134.97,


In [69]:
# set parameters:

params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

In [70]:
# Iterate through list
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    url_base = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(url_base, params = params).json()
    
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
        
    except IndexError:
        print(f"Hotel Not Found -- skipping")

    

Hotel Not Found -- skipping
Hotel Not Found -- skipping
Hotel Not Found -- skipping
Hotel Not Found -- skipping
Hotel Not Found -- skipping
Hotel Not Found -- skipping
Hotel Not Found -- skipping
Hotel Not Found -- skipping
Hotel Not Found -- skipping
Hotel Not Found -- skipping
Hotel Not Found -- skipping


In [72]:
# Save and upload the new DataFrame as WeatherPy_vacation.csv.

output_data_file = "data/WeatherPy_Vacation.csv"
hotel_df.to_csv(output_data_file, index_label = "City_ID")

### Marker Layer Map

In [73]:
# load WeatherPy_Vacation file into vacation_df

vacation_df = pd.read_csv("data/WeatherPy_Vacation.csv")

In [74]:
# add hotel markup to map (copied from VacationPy):

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

locations = vacation_df[["Lat","Lng"]]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))